# Linear models for regression

The linear models are a set of methods intended for regression in which the target value is expected to be a linear combination of the features. In mathematical notation, if $\hat y$ is the predicted value. 
$$\hat{y}(w, x) = w_0 + w_1 x_1 + ... + w_p x_p ,$$ 
where the vector $w=(w_1,\ldots,w_p)$ is `coef_` and $w_0$ is `intercept_`.

## 1. Ordinary Least Squares

<font color = blue>LinearRegression</font> fits a linear model with coefficients $w = (w_1, ..., w_p)$ to minimize the residual sum of squares between the observed targets in the dataset, and the targets predicted by the linear approximation. Mathematically it solves a problem of the form:
$$\min_{w} || X w - y||_2^2$$


<font color = blue>LinearRegression</font> will take in its `fit` method arrays X, y and will store the coefficients $w$ of the linear model in its`coef_` member:

In [1]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit([[0, 0], [1, 1], [2, 2]], [0, 1, 2])

LinearRegression()

In [2]:
reg.coef_

array([0.5, 0.5])

```python
class sklearn.linear_model.LinearRegression(*, fit_intercept=True, normalize=False, copy_X=True, n_jobs=None)
```

Parameters:
- fit_intercept: bool, default=True
    Whether to calculate the intercept for this model. If set to False, no intercept will be used in calculations (i.e. data is expected to be centered).


- normalize: bool, default=False
    This parameter is ignored when `fit_intercept` is set to False. If True, the regressors X will be normalized before regression by subtracting the mean and dividing by the l2-norm. If you wish to standardize, please use <font color = blue>sklearn.preprocessing.StandardScaler</font> before calling `fit` on an estimator with `normalize=False`.


- copy_X: bool, default=True 
    If True, X will be copied; else, it may be overwritten.


- n_jobs: int, default=None
    The number of jobs to use for the computation. This will only provide speedup for n_targets > 1 and sufficient large problems. `None` means 1 unless in a <font color = blue>joblib.parallel_backend</font> context. `-1` means using all processors. 
    
Attributes:
- coef_: array of shape (n_features, ) or (n_targets, n_features)
    Estimated coefficients for the linear regression problem. If multiple targets are passed during the fit (y 2D), this is a 2D array of shape (n_targets, n_features), while if only one target is passed, this is a 1D array of length n_features.
    
- rank_: int
    Rank of matrix `X`. Only available when `X` is dense.
    
- singular_: array of shape (min(X, y),)
    Singular values of `X`. Only available when `X` is dense.
    
- intercept_: float or array of shape (n_targets,)
    Independent term in the linear model. Set to 0.0 if `fit_intercept = False`.

Methods:
- fit(X, y, sample_weight=None): Fit linear model.
    
- get_params(deep=True): Get parameters for this estimator.
    
- predict(X): Predict using the linear model.
    
- score(X, y, sample_weight=None): Return the coefficient of determination R^2 of the prediction.

- set_params(**params): Set the parameters of this estimator.

### Linear Regression Example

This example uses the only the first feature of the `diabetes` dataset. 
The coefficients, the residual sum of squares and the coefficient of determination are also calculated.

In [3]:
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [4]:
# Load the diabetes dataset, which contain 10 attributes: age, sex, body mass index, average blood pressure, and six blood serum measurements
# Target is a quantitative measure of disease progression one year after baseline
diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)

print(diabetes_X.shape)
print(diabetes_y.shape)

(442, 10)
(442,)


In [5]:
# Use only one feature
diabetes_X = diabetes_X[:, 0]
diabetes_X = diabetes_X[:, np.newaxis]

print(diabetes_X.shape)

(442, 1)


In [6]:
# Split the data into training/testing sets
diabetes_X_train = diabetes_X[:-20]
diabetes_X_test = diabetes_X[-20:]

# Split the targets into training/testing sets
diabetes_y_train = diabetes_y[:-20]
diabetes_y_test = diabetes_y[-20:]

print(diabetes_X_train.shape)

(422, 1)


In [7]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(diabetes_X_train, diabetes_y_train)

# Make predictions using the testing set
diabetes_y_pred = regr.predict(diabetes_X_test)

print(diabetes_y_pred.shape)

(20,)


In [8]:
# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(diabetes_y_test, diabetes_y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(diabetes_y_test, diabetes_y_pred))

Coefficients: 
 [306.72757499]
Mean squared error: 5472.26
Coefficient of determination: -0.13


In [9]:
print(regr.score(diabetes_X_test, diabetes_y_test))

-0.13270201630620848


## 2. Polynomial regression: extending linear models with basis functions

One common pattern within machine learning is to use linear models trained on nonlinear functions of the data. This approach maintains the generally fast performance of linear methods, while allowing them to fit a much wider range of data.

For example, a simple linear regression can be extended by constructing <b>polynomial features</b> from the coefficients. In the standard linear regression case, you might have a model that looks like this for two-dimensional data: 
$$ \hat{y}(w, x) = w_0 + w_1 x_1 + w_2 x_2 $$ 
If we want to fit a paraboloid to the data instead of a plane, we can combine the features in second-order polynomials, so that the model looks like this:
$$ \hat{y}(w, x) = w_0 + w_1 x_1 + w_2 x_2 + w_3 x_1 x_2 + w_4 x_1^2 + w_5 x_2^2 $$
The (sometimes surprising) observation is that this is still a linear model: to see this, imagine creating a new set of features
$$ z = [x_1, x_2, x_1 x_2, x_1^2, x_2^2] $$
With this re-labeling of the data, our problem can be written
$$ \hat{y}(w, z) = w_0 + w_1 z_1 + w_2 z_2 + w_3 z_3 + w_4 z_4 + w_5 z_5 $$
We see that the resulting polynomial regression is in the same class of linear models we considered above (i.e. the model is linear in $w$) and can be solved by the same techniques. By considering linear fits within a higher-dimensional space built with these basis functions, the model has the flexibility to fit a much broader range of data.

The class <font color = blue>PolynomialFeatures</font> transforms an input data matrix into a new data matrix of a given degree. It can be used as follows:

In [10]:
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
X = np.arange(6).reshape(3, 2)
print(X)

[[0 1]
 [2 3]
 [4 5]]


In [11]:
poly = PolynomialFeatures(degree=2)
poly.fit_transform(X)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

```python
class sklearn.preprocessing.PolynomialFeatures(degree=2, *, interaction_only=False, include_bias=True, order='C')
```

Parameters:
- degree: integer
    The degree of the polynomial features. Default = 2.


- interaction_only: boolean, default = False
    If true, only interaction features are produced: features that are products of at most `degree` distinct input features (so not `x[1] ** 2`, `x[0] * x[2] ** 3`, etc.).


- include_bias: boolean
    If True (default), then include a bias column, the feature in which all polynomial powers are zero (i.e. a column of ones - acts as an intercept term in a linear model).


- order: str in {‘C’, ‘F’}, default ‘C’
    Order of output array in the dense case. ‘F’ order is faster to compute, but may slow down subsequent estimators.
    
Attributes:
- powers_: array, shape (n_output_features, n_input_features)
    powers_[i, j] is the exponent of the jth input in the ith output.
    
- n_input_features_: int
    The total number of input features.
    
- n_output_features_: int
    The total number of polynomial output features. The number of output features is computed by iterating over all suitably sized combinations of input features.


Methods:
- fit(X, y=None): Compute number of output features.
    
- fit_transform(X, y=None, **fit_params): Fit to data, then transform it.
    
- get_feature_names(input_features=None): Return feature names for output features
    
- get_params(deep=True): Get parameters for this estimator.

- set_params(**params): Set the parameters of this estimator.

- transform(X): Transform data to polynomial features

The features of `X` have been transformed from $[x_1, x_2]$ to $[1, x_1, x_2, x_1^2, x_1 x_2, x_2^2]$, and can now be used within any linear model.

### Polynomial Regression Example
$\phi_i(\mathbf{x})=\mathbf{x}^i$

$\mathbf{w}=\begin{bmatrix}3 \\ -2 \\ 1 \\ -1\end{bmatrix}$

In [16]:
poly = PolynomialFeatures(degree=3)
reg = linear_model.LinearRegression(fit_intercept=False)
# fit to an order-3 polynomial data
x = np.arange(5).reshape(-1, 1)
y = 3 - 2 * x + x ** 2 - x ** 3
x_pol = poly.fit_transform(x)
print(f"X:\n{X}\ny:\n{y}\nx_pol:\n{x_pol}\n")

X:
[[0 1]
 [2 3]
 [4 5]]
y:
[[  3]
 [  1]
 [ -5]
 [-21]
 [-53]]
x_pol:
[[ 1.  0.  0.  0.]
 [ 1.  1.  1.  1.]
 [ 1.  2.  4.  8.]
 [ 1.  3.  9. 27.]
 [ 1.  4. 16. 64.]]



In [15]:
reg.fit(x_pol, y)

LinearRegression(fit_intercept=False)

### Task

Requirement: If we have a linear model
$y=a*x_1+b*x_2+ c *x_1*x_2$
And we have historical data:  
```
X = [[1, 1], [1, 2], [2, 1]],
Y = [4, 5, 6].
```
Compute the estimation of (a, b, c):

From the given information, we can derive that:

$\phi_0(\mathbf{x})=x_1, \ \phi_1(\mathbf{x})=x_2, \ \phi_2(\mathbf{x})=x_1\cdot x_2$

$\mathbf{w}=\begin{bmatrix}a \\ b \\ c\end{bmatrix}$

We could se that
- $M=3$, we make 3 basic operations;
- $N=3$, the data amount is 3.

The $\Phi$ matrix would be:

$\mathbf{\Phi}=\begin{bmatrix} \phi_0(x_1) & \phi_0(x_2) & \cdots & \phi_0(x_N) \\ \phi_1(x_1) & \phi_1(x_2) & \cdots & \phi_1(x_N) \\ \vdots & \vdots & \ddots & \vdots \\ \phi_M(x_1) & \phi_M(x_2) & \cdots & \phi_M(x_N) \end{bmatrix}=\begin{bmatrix}x_{11} & x_{12} & x_{11}x_{12} \\ x_{21} & x_{22} & x_{21}x_{12} \\ x_{31} & x_{32} & x_{31}x_{32}\end{bmatrix}$

In [36]:
y = [[4],[5],[6]]
def phi_0(x1, x2):
    return x1

def phi_1(x1, x2):
    return x2

def phi_2(x1, x2):
    return x1 * x2

phi = []
phi.append(phi_0)
phi.append(phi_1)
phi.append(phi_2)

$\Phi$

In [37]:
big_phi_mat = []
for x_vec in X:
    big_phi_row = []
    for phi_func in phi:
        val = phi_func(x_vec[0], x_vec[1])
        big_phi_row.append(val)
    big_phi_mat.append(big_phi_row)
big_phi_mat

[[np.int64(0), np.int64(1), np.int64(0)],
 [np.int64(2), np.int64(3), np.int64(6)],
 [np.int64(4), np.int64(5), np.int64(20)]]

$\Phi^\top$

In [38]:
big_phi_mat_transpose = np.transpose(big_phi_mat)
big_phi_mat_transpose

array([[ 0,  2,  4],
       [ 1,  3,  5],
       [ 0,  6, 20]])

$(\Phi\Phi^\top)^{-1}$

In [39]:
big_phi_mat_mult_inv = np.linalg.inv(big_phi_mat @ big_phi_mat_transpose)
big_phi_mat_mult_inv

array([[ 4.53125, -2.375  ,  0.71875],
       [-2.375  ,  1.625  , -0.5    ],
       [ 0.71875, -0.5    ,  0.15625]])

$(\Phi\Phi^\top)^{-1}\Phi=\Phi^+$

In [40]:
big_phi_mat_MPInv = big_phi_mat_mult_inv @ big_phi_mat
big_phi_mat_MPInv

array([[-1.8750000e+00,  1.0000000e+00,  1.2500000e-01],
       [ 1.2500000e+00,  8.8817842e-16, -2.5000000e-01],
       [-3.7500000e-01,  0.0000000e+00,  1.2500000e-01]])

$\mathbf{w}_{LS}=(\Phi\Phi^\top)^{-1}\Phi y=\Phi^+y$

In [43]:
w_LS = big_phi_mat_MPInv @ y
w_LS

array([[-1.75],
       [ 3.5 ],
       [-0.75]])